In [1]:
import igl
import scipy
import numpy as np

import nbimporter
from HelperFunctions import *
import os
root_folder = os.getcwd()

In [2]:
"""
General approach for all tests:
Create robust and non-trivial example parameters, often 
using random number generation. Compare the results
of my methods the equivalent scipy or libigl methods
"""

def test_sxs():
    """
    test for SparseMatrix.sparse_times_sparse
    """
    np.random.seed(25)
    
    #size and density of Matrix
    size = 50
    density = .05
    
    #generate random data for A
    A_col = np.random.randint(0, size, int(size * size * density))
    A_row = np.random.randint(0, size, int(size * size * density))
    A_val = np.random.randint(0, 100,int(size * size * density))
    
    #generate random data for B
    B_col = np.random.randint(0, size, int(size * size * density))
    B_row = np.random.randint(0, size, int(size * size * density))
    B_val = np.random.randint(0, 100 , int(size * size * density))

    #run my method with A and B values
    myA = SparseMatrix(A_val,A_col,A_row,size)
    myB = SparseMatrix(B_val,B_col,B_row,size)
    
    myC = SparseMatrix.sparse_times_sparse(myA,myB)
    
    #run scipy version with A and B values
    scipyA = scipy.sparse.coo_matrix((A_val, (A_row, A_col)), shape=(size, size))
    scipyB = scipy.sparse.coo_matrix((B_val, (B_row, B_col)), shape=(size, size))
    
    scipyC = (scipyA @ scipyB).tocoo()
    
    #now compare
    myResults = sorted(zip(myC.r, myC.c, myC.v))
    scipyResults = sorted(zip(scipyC.row, scipyC.col, scipyC.data))
    
    for i,results in enumerate(myResults):
        assert(results[0] == scipyResults[i][0])
        assert(results[1] == scipyResults[i][1])
        assert(results[2] == scipyResults[i][2])

def test_powSp():
    """
    test for SparseMatrix.pow_sparse()
    """
    np.random.seed(64)
    k = 1
    #size and density of Matrix
    size = 50
    density = .05

    #generate random data for A
    A_col = np.random.randint(0, size, int(size * size * density))
    A_row = np.random.randint(0, size, int(size * size * density))
    A_val = np.random.randint(0, 100,int(size * size * density))

    #run my method
    myA = SparseMatrix(A_val,A_col,A_row,size)
    
    myB = myA.pow_sparse(k)
    
    #run scipy version
    scipyA = scipy.sparse.coo_matrix((A_val, (A_row, A_col)), shape=(size, size))
    
    scipyB = scipyA.power(k)
    
    #now compare
    myResults = sorted(zip(myB.r, myB.c, myB.v))
    scipyResults = sorted(zip(scipyB.row, scipyB.col, scipyB.data))
    
    for i,results in enumerate(myResults):
        assert(results[0] == scipyResults[i][0])
        assert(results[1] == scipyResults[i][1])
        assert(results[2] == scipyResults[i][2])

def test_sxd():
    """
    test for SparseMatrix.sparse_times_dense()
    """
    np.random.seed(44)
    
    #size and density of Matrix
    size = 10
    density = .2
    
    #generate random data for A
    A_col = np.random.randint(0, size, int(size * size * density))
    A_row = np.random.randint(0, size, int(size * size * density))
    A_val = np.random.randint(0, 100,int(size * size * density))
    
    #generate random data for B
    B = np.random.rand(size,size)
    
    #run my method
    myA = SparseMatrix(A_val,A_col,A_row,size)
    
    myC = myA.sparse_times_dense(B)
    
    #run scipy method
    scipyA = scipy.sparse.coo_matrix((A_val, (A_row, A_col)), shape=(size, size))
    
    scipyC = scipyA @ B
    
    #compare
    assert(myC.shape == scipyC.shape)
    assert(np.array_equal(myC,scipyC))

def test_sliceSp():
    """
    test for SparseMatrix.slice_sparse
    """
    np.random.seed(89)
    
    #size and density of Matrix
    size = 50
    density = .05
    
    #generate random slicing values
    cols = np.random.randint(0,size, int(size/2))
    rows = np.random.randint(0,size, int(size/2))

    #generate random data for A
    A_col = np.random.randint(0, size, int(size * size * density))
    A_row = np.random.randint(0, size, int(size * size * density))
    A_val = np.random.randint(0, 100,int(size * size * density))
    
    #run my method
    myA = SparseMatrix(A_val,A_col,A_row,size)
    
    myB = myA.slice_sparse(rows,cols)
    
    #run scipy version
    scipyA = scipy.sparse.coo_matrix((A_val, (A_row, A_col)), shape=(size, size))
    
    A_csr = scipyA.tocsr()
    A_row_sliced = A_csr[rows]
    A_final = A_row_sliced[:, cols]
    
    scipyB = A_final.tocoo()
    
    #print(np.setdiff1d(scipyB.row, myB.r), np.setdiff1d(scipyB.col, myB.c))
    
    print(myB.r,myB.c)
    print(myB.v)
    #print(scipyB.row, scipyB.col)
    #now compare
    myResults = sorted(zip(myB.r, myB.c, myB.v))
    scipyResults = sorted(zip(scipyB.row, scipyB.col, scipyB.data))
    
    for i,results in enumerate(myResults):
        #print((results[0],results[1],results[2]),(scipyResults[i][0],scipyResults[i][1],scipyResults[i][2]))
        assert(results[0] == scipyResults[i][0])
        assert(results[1] == scipyResults[i][1])
        assert(results[2] == scipyResults[i][2])

def test_cotLaplace():
    """
    test for BuildLaplacian()
    """
    #for this one I'm gonna use already generated v and f because random will be hard to make valid
    v, f = igl.read_triangle_mesh(os.path.join(root_folder, "decimated-max.obj"))
    
    #my method
    myLaplace: SparseMatrix = buildLaplacian(v,f)
    
    #igl's method
    iglLaplace = igl.cotmatrix(v,f)

    #compare
    iglLaplace = iglLaplace.tocoo()

    myResults = sorted(zip(myLaplace.r, myLaplace.c, myLaplace.v))
    iglResults = sorted(zip(iglLaplace.row, iglLaplace.col, iglLaplace.data))
    #print(len(myResults),len(iglResults))
    for i,results in enumerate(myResults):
        #print((results[0],results[1],results[2]),(iglResults[i][0],iglResults[i][1],iglResults[i][2]))
        assert(results[0] == iglResults[i][0])
        assert(results[1] == iglResults[i][1])
        assert(results[2] == iglResults[i][2])

def test_solve():
    """
    Test for SparseMatrix.solve_sparse()
    """
    np.random.seed(89)
    size = 100000

    # randomly generating A has too high of a chance to make a dependent matrix so hardcoding this one
    # diagonal entries (i,i) = 2.0
    A_row = [i for i in range(size)]
    A_col = [i for i in range(size)]
    A_val = [2.0 for _ in range(size)]

    # lower‐off-diagonal entries (i+1,i) = –1.0
    A_row += [i+1 for i in range(size-1)]
    A_col += [i   for i in range(size-1)]
    A_val += [-1.0 for _ in range(size-1)]

    # upper-off-diagonal entries (i,i+1) = –1.0
    A_row += [i   for i in range(size-1)]
    A_col += [i+1 for i in range(size-1)]
    A_val += [-1.0 for _ in range(size-1)]

    # generate b
    B = np.random.randint(0, size, size) 

    # my solve
    myA   = SparseMatrix(A_val, A_col, A_row, size)
    myAns = SparseMatrix.solve_sparse(myA, B)

    # scipy solve
    scipyA = scipy.sparse.coo_matrix((A_val, (A_row, A_col)), shape=(size, size))
    scipyA = scipyA.tocsr()                
    scipyAns = scipy.sparse.linalg.spsolve(scipyA, B)    
    
    assert np.allclose(myAns, scipyAns)